In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn import preprocessing, model_selection, ensemble, metrics, decomposition
%matplotlib inline

In [2]:
def read_data(filename, sep=';'):
    data = pd.read_csv(filename, header=0, sep=sep, encoding='pt154')
    data = data.rename(index=str, columns={'client_id': '_ID_', 'open_acount_flg': '_VAL_'})
    return data

In [3]:
train = read_data('../data/credit_train.csv')
test = read_data('../data/credit_test.csv')

In [4]:
train.head()

,_ID_,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,"59998,00",10,1.6,"0,770249",GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,"10889,00",6,1.1,"0,248514",GRD,МОСКВА,43000.0,2.0,0.0,0
2,3,M,32,MAR,SPC,"10728,00",12,1.1,"0,459589",SCH,ОБЛ САРАТОВСКАЯ,23000.0,5.0,0.0,0
3,4,F,27,DIV,SPC,"12009,09",12,1.1,"0,362536",GRD,ОБЛ ВОЛГОГРАДСКАЯ,17000.0,2.0,0.0,0
4,5,M,45,MAR,SPC,"16908,89",10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,25000.0,1.0,0.0,0


In [5]:
(train[['living_region', 'open_account_flg']]
.groupby('living_region')
.agg({'open_account_flg': {'mean': np.mean, 'count': len}}))

open_account_flg      
                                                     mean count
living_region                                                  
74                                               0.000000     1
98                                               0.000000     1
АДЫГЕЯ РЕСП                                      0.000000     9
АЛТАЙСКИЙ                                        1.000000     2
АЛТАЙСКИЙ КРАЙ                                   0.375000     8
АМУРСКАЯ ОБЛ                                     0.166667    12
АМУРСКАЯ ОБЛАСТЬ                                 0.100000    20
АО НЕНЕЦКИЙ                                      0.100000    20
АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю         0.128801   559
АО ЯМАЛО-НЕНЕЦКИЙ                                0.134752   141
АОБЛ ЕВРЕЙСКАЯ                                   0.250000     4
АРХАНГЕЛЬСКАЯ                                    0.000000     1
АРХАНГЕЛЬСКАЯ ОБЛ                                0.055556    18
АРХАНГЕЛЬСКАЯ ОБЛАСТЬ                            0.222222     9
АСТРАХАНСКАЯ                                     0.160000    25
АСТРАХАНСКАЯ ОБЛ                                 0.096774    31
АСТРАХАНСКАЯ ОБЛАСТЬ                             0.048951   143
БАШКОРТОСТАН                                     0.115385    26
БАШКОРТОСТАН РЕСП                                0.135338   133
БЕЛГОРОДСКАЯ ОБЛ                                 0.137931    29
БЕЛГОРОДСКАЯ ОБЛАСТЬ                             0.115385    26
БРЯНСКАЯ ОБЛ                                     0.153846    13
БРЯНСКАЯ ОБЛАСТЬ                                 0.285714     7
БРЯНСКИЙ                                         0.000000     1
БУРЯТИЯ                                          0.000000    18
БУРЯТИЯ РЕСП                                     0.518519    27
ВЛАДИМИРСКАЯ ОБЛ                                 0.161290    31
ВЛАДИМИРСКАЯ ОБЛАСТЬ                             0.080000    25
ВОЛГОГРАДСКАЯ ОБЛ                                0.333333    30
ВОЛГОГРАДСКАЯ ОБЛАСТЬ                            0.157895    38
...                                                   ...   ...
ТУЛЬСКАЯ ОБЛ                                     0.205069   868
ТУЛЬСКАЯ ОБЛАСТЬ                                 0.150943   424
ТЫВА РЕСП                                        0.364796   392
ТЮМЕНСКАЯ                                        0.000000     1
ТЮМЕНСКАЯ ОБЛ                                    0.263069  1186
ТЮМЕНСКАЯ ОБЛАСТЬ                                0.174342   608
УДМУРТСКАЯ РЕСП                                  0.238606   746
УДМУРТСКАЯ РЕСПУБЛИКА                            0.107266   289
УЛЬЯНОВСКАЯ ОБЛ                                  0.195841   577
УЛЬЯНОВСКАЯ ОБЛАСТЬ                              0.086567   335
ХАБАРОВСКИЙ КРАЙ                                 0.212466  1492
ХАКАСИЯ                                          1.000000     1
ХАКАСИЯ РЕСП                                     0.223214   336
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА         0.221125  2329
ХАНТЫ-МАНСИЙСКИЙ АО                              0.172043  1488
ЧЕЛЯБИНСКАЯ ОБЛ                                  0.202135  2904
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ                              0.162420  1570
ЧЕЧЕНСКАЯ РЕСП                                   0.363636    11
ЧЕЧЕНСКАЯ РЕСПУБЛИКА                             0.333333     3
ЧИТИНСКАЯ ОБЛ                                    0.235294    17
ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -                   0.218182    55
ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП                         0.235294    68
ЧУВАШСКАЯ РЕСП                                   0.000000     1
ЧУВАШСКАЯ РЕСПУБЛИКА                             0.173759   282
ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ                   0.200389   514
ЧУКОТСКИЙ АО                                     0.750000    32
ЭВЕНКИЙСКИЙ АО                                   1.000000     1
ЯМАЛО-НЕНЕЦКИЙ АО                                0.201324  1813
ЯРОСЛАВСКАЯ ОБЛ                                  0.150635   551
ЯРОСЛАВСК

In [6]:
cities = \
{np.nan: ('BAD NAME', 0, 0, 0),
 'ОБЛ КУРСКАЯ': ('КУРСКАЯ', 1, 1, 1),
 'ПЕРМСКАЯ ОБЛ': ('ПЕРМСКАЯ', 2, 1, 1),
 'ОБЛ ПЕНЗЕНСКАЯ': ('ПЕНЗЕНСКАЯ', 1, 1, 1),
 'РЕСП ИНГУШЕТИЯ': ('ИНГУШЕТИЯ', 1, 1, 2),
 'САХАЛИНСКАЯ ОБЛАСТЬ': ('САХАЛИНСКАЯ', 2, 2, 1),
 'ЯМАЛО-НЕНЕЦКИЙ АО': ('ЯМАЛО-НЕНЕЦКИЙ', 2, 1, 3),
 'Г. МОСКВА': ('МОСКВА', 1, 1, 4),
 'СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ РЕСП': ('СЕВЕРНАЯ ОСЕТИЯ', 2, 1, 2),
 'РЕСП ЧЕЧЕНСКАЯ': ('ЧЕЧЕНСКАЯ', 1, 1, 2),
 'ТЮМЕНСКАЯ ОБЛАСТЬ': ('ТЮМЕНСКАЯ', 2, 2, 1),
 'УДМУРТСКАЯ РЕСПУБЛИКА': ('УДМУРТСКАЯ', 2, 2, 2),
 'ВОЛГОГРАДСКАЯ ОБЛАСТЬ': ('ВОЛГОГРАДСКАЯ', 2, 2, 1),
 'ГОРЬКОВСКАЯ ОБЛ': ('ГОРЬКОВСКАЯ', 2, 1, 1),
 'ОБЛ ОРЛОВСКАЯ': ('ОРЛОВСКАЯ', 1, 1, 1),
 'СВЕРДЛОВСКАЯ ОБЛАСТЬ': ('СВЕРДЛОВСКАЯ', 2, 2, 1),
 'КРАЙ ПЕРМСКИЙ': ('ПЕРМСКИЙ', 1, 2, 5),
 'КРАСНОЯРСКИЙ КРАЙ': ('КРАСНОЯРСКИЙ', 2, 2, 5),
 'СЕВ. ОСЕТИЯ - АЛАНИЯ': ('СЕВЕВРНАЯ ОСЕТИЯ - АЛАНИЯ', 0, 0, 0),
 'ОБЛ СМОЛЕНСКАЯ': ('СМОЛЕНСКАЯ', 1, 1, 1),
 'КАБАРДИНО-БАЛКАРСКАЯ': ('КАБАРДИНО-БАЛКАРСКАЯ', 0, 0, 0),
 'НОВГОРОДСКАЯ ОБЛАСТЬ': ('НОВГОРОДСКАЯ', 2, 2, 1),
 'ОБЛ ИВАНОВСКАЯ': ('ИВАНОВСКАЯ', 1, 1, 1),
 'САНКТ-ПЕТЕРБУРГ Г': ('САНКТ-ПЕТЕРБУРГ', 2, 1, 4),
 'РЯЗАНСКАЯ ОБЛАСТЬ': ('РЯЗАНСКАЯ', 2, 2, 1),
 'ОБЛ КИРОВСКАЯ': ('КИРОВСКАЯ', 1, 1, 1),
 'ЛЕНИНГРАДСКАЯ': ('ЛЕНИНГРАДСКАЯ', 0, 0, 0),
 'ХАНТЫ-МАНСИЙСКИЙ АО': ('ХАНТЫ-МАНСИЙСКИЙ', 2, 1, 3),
 'ОБЛ ЛЕНИНГРАДСКАЯ': ('ЛЕНИНГРАДСКАЯ', 1, 1, 1),
 'РЕСП МАРИЙ ЭЛ': ('МАРИЙ ЭЛ', 1, 1, 2),
 'БУРЯТИЯ': ('БУРЯТИЯ', 0, 0, 0),
 'ОБЛ ВОЛГОГРАДСКАЯ': ('ВОЛГОГРАДСКАЯ', 1, 1, 1),
 'РЕСП АДЫГЕЯ': ('АДЫГЕЯ', 1, 1, 2),
 'УДМУРТСКАЯ РЕСП': ('УДМУРТСКАЯ', 2, 1, 2),
 'ОБЛ САМАРСКАЯ': ('САМАРСКАЯ', 1, 1, 1),
 'ЕВРЕЙСКАЯ АОБЛ': ('ЕВРЕЙСКАЯ', 2, 1, 3),
 'ОБЛ РЯЗАНСКАЯ': ('РЯЗАНСКАЯ', 1, 1, 1),
 'КАРАЧАЕВО-ЧЕРКЕССКАЯ': ('КАРАЧАЕВО-ЧЕРКЕССКАЯ', 0, 0, 0),
 'ОБЛ САХАЛИНСКАЯ': ('САХАЛИНСКАЯ', 1, 1, 1),
 'ОБЛ ЛИПЕЦКАЯ': ('ЛИПЕЦКАЯ', 1, 1, 1),
 'РЕСПУБЛИКА МОРДОВИЯ': ('МОРДОВИЯ', 1, 2, 2),
 'РОСТОВСКАЯ ОБЛ': ('РОСТОВСКАЯ', 2, 1, 1),
 'КАМЧАТСКАЯ ОБЛАСТЬ': ('КАМЧАТСКАЯ', 2, 2, 1),
 'РЕСП КАРАЧАЕВО-ЧЕРКЕССКАЯ': ('КАРАЧАЕВО-ЧЕРКЕССКАЯ', 1, 1, 2),
 'АЛТАЙСКИЙ КРАЙ': ('АЛТАЙСКИЙ', 2, 2, 5),
 'ОРЛОВСКАЯ ОБЛАСТЬ': ('ОРЛОВСКАЯ', 2, 2, 1),
 'КРАЙ КАМЧАТСКИЙ': ('КАМЧАТСКИЙ', 1, 2, 5),
 'РЕСП КАРЕЛИЯ': ('КАРЕЛИЯ', 1, 1, 2),
 'КАЛУЖСКАЯ ОБЛ': ('КАЛУЖСКАЯ', 2, 1, 1),
 'ХАБАРОВСКИЙ КРАЙ': ('ХАБАРОВСКИЙ', 2, 2, 5),
 'АСТРАХАНСКАЯ ОБЛ': ('АСТРАХАНСКАЯ', 2, 1, 1),
 'БАШКОРТОСТАН': ('БАШКОРТОСТАН', 0, 0, 0),
 'ОБЛ МОСКОВСКАЯ': ('МОСКОВСКАЯ', 1, 1, 1),
 'Г. САНКТ-ПЕТЕРБУРГ': ('САНКТ-ПЕТЕРБУРГ', 1, 1, 4),
 'ОБЛ СВЕРДЛОВСКАЯ': ('СВЕРДЛОВСКАЯ', 1, 1, 1),
 'КАМЧАТСКИЙ КРАЙ': ('КАМЧАТСКИЙ', 2, 2, 2),
 'САХА /ЯКУТИЯ/ РЕСП': ('САХА', 2, 1, 2),
 'САРАТОВСКАЯ ОБЛАСТЬ': ('САРАТОВСКАЯ', 2, 2, 1),
 'КРАЙ АЛТАЙСКИЙ': ('АЛТАЙСКИЙ', 1, 2, 5),
 'КУРСКАЯ ОБЛАСТЬ': ('КУРСКАЯ', 2, 2, 1),
 'ВЛАДИМИРСКАЯ ОБЛАСТЬ': ('ВЛАДИМИРСКАЯ', 2, 2, 1),
 'АСТРАХАНСКАЯ': ('АСТРАХАНСКАЯ', 0, 0, 0),
 'ОМСКАЯ ОБЛ': ('ОМСКАЯ', 2, 1, 1),
 'ТУЛЬСКАЯ ОБЛ': ('ТУЛЬСКАЯ', 2, 1, 1),
 'САМАРСКАЯ ОБЛ': ('САМАРСКАЯ', 2, 1, 1),
 'ОБЛ МАГАДАНСКАЯ': ('МАГАДАНСКАЯ', 1, 1, 1),
 'КАРЕЛИЯ РЕСП': ('КАРЕЛИЯ', 2, 1, 2),
 'КРАСНОДАРСКИЙ КРАЙ': ('КРАСНОДАРСКИЙ', 2, 2, 5),
 'ОБЛ ТАМБОВСКАЯ': ('ТАМБОВСКАЯ', 1, 1, 1),
 'ПЕНЗЕНСКАЯ ОБЛ': ('ПЕНЗЕНСКАЯ', 2, 1, 1),
 'ПСКОВСКАЯ ОБЛАСТЬ': ('ПСКОВСКАЯ', 2, 2, 1),
 'ТАТАРСТАН РЕСП': ('ТАТАРСТАН', 2, 1, 2),
 'ИРКУТСКАЯ ОБЛАСТЬ': ('ИРКУТСКАЯ', 2, 2, 2),
 'ТАМБОВСКАЯ ОБЛАСТЬ': ('ТАМБОВСКАЯ', 2, 2, 2),
 'ОБЛ КАЛИНИНГРАДСКАЯ': ('КАЛИНИНГРАДСКАЯ', 1, 1, 1),
 'АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю': ('ХАНТЫ-МАНСИЙСКИЙ', 1, 1, 3),
 'ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -': ('ЧУВАШСКАЯ', 2, 1, 2),
 'КРАЙ ХАБАРОВСКИЙ': ('ХАБАРОВСКИЙ', 1, 2, 5),
 'ОБЛ ПСКОВСКАЯ': ('ПСКОВСКАЯ', 1, 1, 1),
 'ЛИПЕЦКАЯ ОБЛ': ('ЛИПЕЦКАЯ', 2, 1, 1),
 'ОБЛ МУРМАНСКАЯ': ('МУРМАНСКАЯ', 1, 1, 1),
 'ЛИПЕЦКАЯ ОБЛАСТЬ': ('ЛИПЕЦКАЯ', 2, 2, 1),
 'РЕСП. БАШКОРТОСТАН': ('БАШКОРТОСТАН', 1, 1, 2),
 'ОБЛ КАЛУЖСКАЯ': ('КАЛУЖСКАЯ', 1, 1, 1),
 'ТАМБОВСКАЯ ОБЛ': ('ТАМБОВСКАЯ', 2, 1, 1),
 'МОСКВА Г': ('МОСКВА', 2, 1, 4),
 'ВЛАДИМИРСКАЯ ОБЛ': ('ВЛАДИМИРСКАЯ', 2, 1, 1),
 'ДАГЕСТАН РЕСП': ('ДАГЕСТАН', 2, 1, 2),
 'ТЫВА РЕСП': ('ТЫВА', 2, 1, 2),
 'НИЖЕГОРОДСКАЯ ОБЛАСТЬ': ('НИЖЕГОРОДСКАЯ', 2, 2, 1),
 'ВОЛОГОДСКАЯ ОБЛ': ('ВОЛОГОДСКАЯ', 2, 1, 1),
 'МАРИЙ ЭЛ РЕСП': ('МАРИЙ ЭЛ', 2, 1, 2),
 'ХАКАСИЯ РЕСП': ('ХАКАСИЯ', 2, 1, 2),
 'ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ': ('ЧУВАШСКАЯ', 2, 2, 2),
 'ОБЛ КУРГАНСКАЯ': ('КУРГАНСКАЯ', 1, 1, 1),
 'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА': ('ХАНТЫ-МАНСИЙСКИЙ', 2, 2, 3),
 'РЕСП.БАШКОРТОСТАН': ('БАШКОРТОСТАН', 1, 1, 2),
 'ОБЛ ВОЛОГОДСКАЯ': ('ВОЛОГОДСКАЯ', 1, 1, 1),
 'РЕСП КАБАРДИНО-БАЛКАРСКАЯ': ('КАБАРДИНО-БАЛКАРСКАЯ', 1, 1, 2),
 'ЧУКОТСКИЙ АО': ('ЧУКОТСКИЙ', 2, 1, 3),
 'НОВОСИБИРСКАЯ ОБЛАСТЬ': ('НОВОСИБИРСКАЯ', 2, 1, 1),
 'РЕСПУБЛИКА БУРЯТИЯ': ('БУРЯТИЯ', 1, 2, 2),
 'КАЛМЫКИЯ РЕСП': ('КАЛМЫКИЯ', 2, 1, 2),
 'КОМИ РЕСП': ('КОМИ', 2, 1, 2),
 'ОБЛ ТОМСКАЯ': ('ТОМСКАЯ', 1, 1, 1),
 'БАШКОРТОСТАН РЕСП': ('БАШКОРТОСТАН', 2, 1, 2),
 'КАРАЧАЕВО-ЧЕРКЕССКАЯ РЕСП': ('КАРАЧАЕВО-ЧЕРКЕССКАЯ', 2, 1, 2),
 'ЧИТИНСКАЯ ОБЛ': ('ЧИТИНСКАЯ', 2, 1, 1),
 'ОБЛ ТУЛЬСКАЯ': ('ТУЛЬСКАЯ', 1, 1, 1),
 'ОБЛ ВЛАДИМИРСКАЯ': ('ВЛАДИМИРСКАЯ', 1, 1, 1),
 'РЕСПУБЛИКА ТАТАРСТАН': ('ТАТАРСТАН', 1, 2, 2),
 'ОБЛ РОСТОВСКАЯ': ('РОСТОВСКАЯ', 1, 1, 1),
 'ЧЕЛЯБИНСКАЯ ОБЛ': ('ЧЕЛЯБИНСКАЯ', 2, 1, 1),
 'ОБЛ ЯРОСЛАВСКАЯ': ('ЯРОСЛАВСКАЯ', 1, 1, 1),
 'АО ЯМАЛО-НЕНЕЦКИЙ': ('ЯМАЛО-НЕНЕЦКИЙ', 1, 1, 3),
 'СТАВРОПОЛЬСКИЙ': ('СТАВРОПОЛЬСКИЙ', 0, 0, 0),
 'СТАВРОПОЛЬСКИЙ КРАЙ': ('СТАВРОПОЛЬСКИЙ', 2, 2, 5),
 'ИНГУШЕТИЯ РЕСП': ('ИНГУШЕТИЯ', 2, 1, 2),
 'АРХАНГЕЛЬСКАЯ': ('АРХАНГЕЛЬСКАЯ', 0, 0, 0),
 'ТОМСКАЯ ОБЛ': ('ТОМСКАЯ', 2, 1, 1),
 'КАРЕЛИЯ': ('КАРЕЛИЯ', 0, 0, 0),
 'КУРСКАЯ ОБЛ': ('КУРСКАЯ', 2, 1, 1),
 'ВОЛОГОДСКАЯ ОБЛАСТЬ': ('ВОЛОГОДСКАЯ', 2, 2, 1),
 'РЕСП САХА /ЯКУТИЯ/': ('САХА', 1, 1, 2),
 'КОСТРОМСКАЯ ОБЛ': ('КОСТРОМСКАЯ', 2, 1, 1),
 'БРЯНСКАЯ ОБЛ': ('БРЯНСКАЯ', 2, 1, 1),
 'АМУРСКАЯ ОБЛ': ('АМУРСКАЯ', 2, 1, 1),
 'РОСТОВСКАЯ ОБЛАСТЬ': ('РОСТОВСКАЯ', 2, 2, 1),
 'ОБЛ НОВОСИБИРСКАЯ': ('НОВОСИБИРСКАЯ', 1, 1, 1),
 'КАЛИНИНГРАДСКАЯ ОБЛ': ('КАЛИНИНГРАДСКАЯ', 2, 1, 1),
 'ОБЛ КОСТРОМСКАЯ': ('КОСТРОМСКАЯ', 1, 1, 1),
 'ТВЕРСКАЯ ОБЛ': ('ТВЕРСКАЯ', 2, 1, 1),
 'ОБЛ БРЯНСКАЯ': ('БРЯНСКАЯ', 1, 1, 1),
 'ВОЛОГОДСКАЯ': ('ВОЛОГОДСКАЯ', 0, 0, 0),
 'МОСКОВСКАЯ ОБЛ': ('МОСКОВСКАЯ', 2, 1, 1),
 'ОБЛ АРХАНГЕЛЬСКАЯ': ('АРХАНГЕЛЬСКАЯ', 1, 1, 1),
 'САМАРСКАЯ ОБЛАСТЬ': ('САМАРСКАЯ', 2, 2, 1),
 'НОВОСИБИРСКАЯ ОБЛ': ('НОВОСИБИРСКАЯ', 2, 1, 1),
 'КРАЙ ЗАБАЙКАЛЬСКИЙ': ('ЗАБАЙКАЛЬСКИЙ', 1, 2, 5),
 'БРЯНСКАЯ ОБЛАСТЬ': ('БРЯНСКАЯ', 2, 2, 1),
 'КРАЙ ПРИМОРСКИЙ': ('ПРИМОРСКИЙ', 1, 2, 5),
 'ОРЛОВСКАЯ ОБЛ': ('ОРЛОВСКАЯ', 2, 1, 1),
 'ОБЛ ЧЕЛЯБИНСКАЯ': ('ЧЕЛЯБИНСКАЯ', 1, 1, 1),
 'БЕЛГОРОДСКАЯ ОБЛАСТЬ': ('БЕЛГОРОДСКАЯ', 2, 2, 1),
 'РЕСП БАШКОРТОСТАН': ('БАШКОРТОСТАН', 1, 1, 2),
 'ОБЛ УЛЬЯНОВСКАЯ': ('УЛЬЯНОВСКАЯ', 1, 1, 1),
 'ОБЛ САРАТОВСКАЯ': ('САРАТОВСКАЯ', 1, 1, 1),
 'РЕСП СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ': ('СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ', 1, 1, 2),
 'САМАРСКАЯ': ('САМАРСКАЯ', 0, 0, 0),
 'РЕСП КОМИ': ('КОМИ', 1, 1, 2),
 'ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП': ('ЧУВАШСКАЯ', 2, 1, 2),
 'РЕСП ТЫВА': ('ТЫВА', 1, 1, 2),
 'ОМСКАЯ ОБЛАСТЬ': ('ОМСКАЯ', 2, 2, 1),
 'ПРИМОРСКИЙ КРАЙ': ('ПРИМОРСКИЙ', 2, 2, 5),
 'ТВЕРСКАЯ ОБЛАСТЬ': ('ТВЕРСКАЯ', 2, 2, 1),
 'ЗАБАЙКАЛЬСКИЙ КРАЙ': ('ЗАБАЙКАЛЬСКИЙ', 2, 2, 5),
 'АОБЛ ЕВРЕЙСКАЯ': ('ЕВРЕЙСКАЯ', 1, 1, 3),
 'ИРКУТСКАЯ ОБЛ': ('ИРКУТСКАЯ', 2, 1, 1),
 'КОСТРОМСКАЯ ОБЛАСТЬ': ('КОСТРОМСКАЯ', 2, 2, 1),
 'АЛТАЙСКИЙ': ('АЛТАЙСКИЙ', 0, 0, 0),
 'ОРЕНБУРГСКАЯ ОБЛ': ('ОРЕНБУРГСКАЯ', 2, 1, 1),
 'НЕНЕЦКИЙ АО': ('НЕНЕЦКИЙ', 2, 1, 3),
 'МУРМАНСКАЯ ОБЛ': ('МУРМАНСКАЯ', 2, 1, 1),
 'НОВГОРОДСКАЯ ОБЛ': ('НОВГОРОДСКАЯ', 2, 1, 1),
 'ЧЕЧЕНСКАЯ РЕСП': ('ЧЕЧЕНСКАЯ', 2, 1, 2),
 'ТУЛЬСКАЯ ОБЛАСТЬ': ('ТУЛЬСКАЯ', 2, 2, 1),
 'КЕМЕРОВСКАЯ ОБЛАСТЬ': ('КЕМЕРОВСКАЯ', 2, 2, 1),
 'ОБЛ ОМСКАЯ': ('ОМСКАЯ', 1, 1, 1),
 'ВОРОНЕЖСКАЯ ОБЛАСТЬ': ('ВОРОНЕЖСКАЯ', 2, 2, 1),
 'ОБЛ ТВЕРСКАЯ': ('ТВЕРСКАЯ', 1, 1, 1),
 'СМОЛЕНСКАЯ ОБЛ': ('СМОЛЕНСКАЯ', 2, 1, 1),
 'РЕСП ДАГЕСТАН': ('ДАГЕСТАН', 1, 1, 2),
 'КРАСНОДАРСКИЙ': ('КРАСНОДАРСКИЙ', 0, 0, 0),
 'РЕСП УДМУРТСКАЯ': ('УДМУРТСКАЯ', 1, 1, 2),
 'РЯЗАНСКАЯ ОБЛ': ('РЯЗАНСКАЯ', 2, 1, 1),
 'МУРМАНСКАЯ ОБЛАСТЬ': ('МУРМАНСКАЯ', 2, 2, 1),
 'ТОМСКАЯ ОБЛАСТЬ': ('ТОМСКАЯ', 2, 2, 1),
 'БЕЛГОРОДСКАЯ ОБЛ': ('БЕЛГОРОДСКАЯ', 2, 1, 1),
 'САРАТОВСКАЯ ОБЛ': ('САРАТОВСКАЯ', 2, 1, 1),
 'ВОРОНЕЖСКАЯ ОБЛ': ('ВОРОНЕЖСКАЯ', 2, 1, 1),
 'КУРГАНСКАЯ ОБЛ': ('КУРГАНСКАЯ', 2, 1, 1),
 'КАЛУЖСКАЯ ОБЛАСТЬ': ('КАЛУЖСКАЯ', 2, 2, 1),
 'РЕСПУБЛИКА ДАГЕСТАН': ('ДАГЕСТАН', 1, 2, 2),
 'ПЕНЗЕНСКАЯ ОБЛАСТЬ': ('ПЕНЗЕНСКАЯ', 2, 2, 1),
 'РЕСПУБЛИКА ТЫВА': ('ТЫВА', 1, 2, 2),
 'ЧУВАШСКАЯ РЕСПУБЛИКА': ('ЧУВАШСКАЯ', 2, 2, 2),
 'ЧЕЛЯБИНСКАЯ ОБЛАСТЬ': ('ЧЕЛЯБИНСКАЯ', 2, 2, 1),
 'РЕСП ХАКАСИЯ': ('ХАКАСИЯ', 1, 1, 2),
 'МОРДОВИЯ РЕСП': ('МОРДОВИЯ', 2, 1, 2),
 'ОБЛ ОРЕНБУРГСКАЯ': ('ОРЕНБУРГСКАЯ', 1, 1, 1),
 'РЕСПУБЛИКА КАЛМЫКИЯ': ('КАЛМЫКИЯ', 1, 2, 2),
 'АО НЕНЕЦКИЙ': ('НЕНЕЦКИЙ', 1, 1, 3),
 'ОБЛ БЕЛГОРОДСКАЯ': ('БЕЛГОРОДСКАЯ', 1, 1, 1),
 'КРАЙ СТАВРОПОЛЬСКИЙ': ('СТАВРОПОЛЬСКИЙ', 1, 2, 5),
 'ОБЛ АМУРСКАЯ': ('АМУРСКАЯ', 1, 1, 1),
 'МОСКОВСКАЯ ОБЛАСТЬ': ('МОСКОВСКАЯ', 2, 2, 1),
 'ЯРОСЛАВСКАЯ ОБЛ': ('ЯРОСЛАВСКАЯ', 2, 1, 1),
 'ЧЕЧЕНСКАЯ РЕСПУБЛИКА': ('ЧЕЧЕНСКАЯ', 2, 2, 2),
 'КУРГАНСКАЯ ОБЛАСТЬ': ('КУРГАНСКАЯ', 2, 2, 1),
 'УЛЬЯНОВСКАЯ ОБЛАСТЬ': ('УЛЬЯНОВСКАЯ', 2, 2, 1),
 'АМУРСКАЯ ОБЛАСТЬ': ('АМУРСКАЯ', 2, 2, 1),
 'ОБЛ КЕМЕРОВСКАЯ': ('КЕМЕРОВСКАЯ', 1, 1, 1),
 'ОРЕНБУРГСКАЯ ОБЛАСТЬ': ('ОРЕНБУРГСКАЯ', 2, 2, 1),
 'ОБЛ ТЮМЕНСКАЯ': ('ТЮМЕНСКАЯ', 1, 1, 1),
 'КИРОВСКАЯ ОБЛ': ('КИРОВСКАЯ', 2, 1, 1),
 'ОБЛ ИРКУТСКАЯ': ('ИРКУТСКАЯ', 1, 1, 1),
 'КЕМЕРОВСКАЯ ОБЛ': ('КЕМЕРОВСКАЯ', 2, 1, 1),
 'РЕСП МОРДОВИЯ': ('МОРДОВИЯ', 1, 1, 2),
 'ТЮМЕНСКАЯ ОБЛ': ('ТЮМЕНСКАЯ', 2, 1, 1),
 'ОБЛ ВОРОНЕЖСКАЯ': ('ВОРОНЕЖСКАЯ', 1, 1, 1),
 'ОБЛ НОВГОРОДСКАЯ': ('НОВГОРОДСКАЯ', 1, 1, 1),
 'РЕСПУБЛИКА МАРИЙ ЭЛ': ('МАРИЙ ЭЛ', 1, 2, 2),
 'РЕСПУБЛИКА ХАКАСИЯ': ('ХАКАСИЯ', 1, 2, 2),
 'РЕСП КАЛМЫКИЯ': ('КАЛМЫКИЯ', 1, 1, 2),
 'ВОЛГОГРАДСКАЯ ОБЛ': ('ВОЛГОГРАДСКАЯ', 2, 1, 1),
 'ЯРОСЛАВСКАЯ ОБЛАСТЬ': ('ЯРОСЛАВСКАЯ', 2, 2, 1),
 'ИВАНОВСКАЯ ОБЛ': ('ИВАНОВСКАЯ', 2, 1, 1),
 'РЕСП АЛТАЙ': ('АЛТАЙ', 1, 1, 2),
 'УЛЬЯНОВСКАЯ ОБЛ': ('УЛЬЯНОВСКАЯ', 2, 1, 1),
 'РЕСПУБЛИКА АДЫГЕЯ': ('АДЫГЕЯ', 1, 2, 2),
 'РЕСП ТАТАРСТАН': ('ТАТАРСТАН', 1, 1, 2),
 'РЕСП. КОМИ': ('КОМИ', 1, 1, 2),
 'АРХАНГЕЛЬСКАЯ ОБЛАСТЬ': ('АРХАНГЕЛЬСКАЯ', 2, 2, 1),
 'СВЕРДЛОВСКАЯ ОБЛ': ('СВЕРДЛОВСКАЯ', 2, 1, 1),
 'ОБЛ АСТРАХАНСКАЯ': ('АСТРАХАНСКАЯ', 1, 1, 1),
 'ПСКОВСКАЯ ОБЛ': ('ПСКОВСКАЯ', 2, 1, 1),
 'КИРОВСКАЯ ОБЛАСТЬ': ('КИРОВСКАЯ', 2, 2, 1),
 'КАБАРДИНО-БАЛКАРСКАЯ РЕСП': ('КАБАРДИНО-БАЛКАРСКАЯ', 2, 1, 2),
 'ЕВРЕЙСКАЯ АВТОНОМНАЯ': ('ЕВРЕЙСКАЯ', 0, 0, 0),
 'АДЫГЕЯ РЕСП': ('АДЫГЕЯ', 2, 1, 2),
 'САХАЛИНСКАЯ ОБЛ': ('САХАЛИНСКАЯ', 2, 1, 1),
 'ЛЕНИНГРАДСКАЯ ОБЛ': ('ЛЕНИНГРАДСКАЯ', 2, 1, 1),
 'РЕСП. САХА (ЯКУТИЯ)': ('САХА', 1, 1, 2),
 'ПЕРМСКИЙ КРАЙ': ('ПЕРМСКИЙ', 2, 2, 5),
 'МОСКОВСКАЯ': ('МОСКОВСКАЯ', 0, 0, 0),
 'РЕСПУБЛИКА КАРЕЛИЯ': ('КАРЕЛИЯ', 1, 2, 2),
 'НИЖЕГОРОДСКАЯ ОБЛ': ('НИЖЕГОРОДСКАЯ', 2, 1, 1),
 'СМОЛЕНСКАЯ ОБЛАСТЬ': ('СМОЛЕНСКАЯ', 2, 2, 1),
 'КАЛИНИНГРАДСКАЯ ОБЛ.': ('КАЛИНИНГРАДСКАЯ', 2, 1, 1),
 'РЕСПУБЛИКА КОМИ': ('КОМИ', 1, 2, 2),
 'ОБЛ НИЖЕГОРОДСКАЯ': ('НИЖЕГОРОДСКАЯ', 1, 1, 1),
 'КРАЙ КРАСНОДАРСКИЙ': ('КРАСНОДАРСКИЙ', 1, 2, 5),
 'РЕСП БУРЯТИЯ': ('БУРЯТИЯ', 1, 1, 2),
 'САНКТ-ПЕТЕРБУРГ': ('САНКТ-ПЕТЕРБУРГ', 0, 0, 0),
 'Г МОСКВА': ('МОСКВА', 1, 1, 4),
 'ИВАНОВСКАЯ ОБЛАСТЬ': ('ИВАНОВСКАЯ', 2, 2, 1),
 'АСТРАХАНСКАЯ ОБЛАСТЬ': ('АСТРАХАНСКАЯ', 2, 2, 1),
 'АРХАНГЕЛЬСКАЯ ОБЛ': ('АРХАНГЕЛЬСКАЯ', 2, 1, 1),
 'ЛЕНИНГРАДСКАЯ ОБЛАСТЬ': ('ЛЕНИНГРАДСКАЯ', 2, 2, 1),
 'МОСКВА': ('МОСКВА', 0, 0, 0),
 'БУРЯТИЯ РЕСП': ('БУРЯТИЯ', 2, 1, 2),
 'КРАЙ КРАСНОЯРСКИЙ': ('КРАСНОЯРСКИЙ', 1, 2, 5),
 '74': ('BAD NAME', 0, 0, 0),
 '98': ('BAD NAME', 0, 0, 0),
 'БРЯНСКИЙ': ('БРЯНСКИЙ', 0, 0, 0),
 'ВОЛОГОДСКАЯ ОБЛ.': ('ВОЛОГОДСКАЯ', 2, 1, 1),
 'Г.МОСКВА': ('МОСКВА', 1, 1, 4),
 'Г.ОДИНЦОВО МОСКОВСКАЯ ОБЛ': ('МОСКОВСКАЯ', 2, 1, 1),
 'ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н': ('BAD NAME', 0, 0, 0),
 'ДАЛЬНИЙ ВОСТОК': ('ДАЛЬНИЙ ВОСТОК', 0, 0, 0),
 'КАЛМЫКИЯ': ('КАЛМЫКИЯ', 0, 0, 0),
 'КАЛУЖСКАЯ': ('КАЛУЖСКАЯ', 0, 0, 0),
 'КЕМЕРОВСКАЯ': ('КЕМЕРОВСКАЯ', 0, 0, 0),
 'КОМИ': ('КОМИ', 0, 0, 0),
 'КРАЙ. КРАСНОЯРСКИЙ': ('КРАСНОЯРСКИЙ', 1, 2, 5),
 'КРАЙ. ПЕРМСКИЙ': ('ПЕРМСКИЙ', 1, 2, 5),
 'КРАЙ. СТАВРОПОЛЬСКИЙ': ('СТАВРОПОЛЬСКИЙ', 1, 2, 5),
 'КРАЙ.ПЕРМСКИЙ': ('ПЕРМСКИЙ', 1, 2, 5),
 'МАГАДАНСКАЯ ОБЛАСТЬ': ('МАГАДАНСКАЯ', 2, 2, 1),
 'МОСКВОСКАЯ ОБЛ': ('МОСКВОСКАЯ', 2, 1, 1),
 'МЫТИЩИНСКИЙ Р-Н': ('МЫТИЩИНСКИЙ', 0, 0, 0),
 'НОВОСИБИРСКАЯ': ('НОВОСИБИРСКАЯ', 0, 0, 0),
 'ОБЛ. БЕЛГОРОДСКАЯ': ('БЕЛГОРОДСКАЯ', 1, 1, 1),
 'ОБЛ. ВЛАДИМИРСКАЯ': ('ВЛАДИМИРСКАЯ', 1, 1, 1),
 'ОБЛ. КИРОВСКАЯ': ('КИРОВСКАЯ', 1, 1, 1),
 'ОБЛ. КУРГАНСКАЯ': ('КУРГАНСКАЯ', 1, 1, 1),
 'ОБЛ. ЛИПЕЦКАЯ': ('ЛИПЕЦКАЯ', 1, 1, 1),
 'ОБЛ. МУРМАНСКАЯ': ('МУРМАНСКАЯ', 1, 1, 1),
 'ОБЛ. НОВОСИБИРСКАЯ': ('НОВОСИБИРСКАЯ', 1, 1, 1),
 'ОБЛ. СВЕРДЛОВСКАЯ': ('СВЕРДЛОВСКАЯ', 1, 1, 1),
 'ОБЛ. ЧЕЛЯБИНСКАЯ': ('ЧЕЛЯБИНСКАЯ', 1, 1, 1),
 'ОБЛ.МОСКОВСКАЯ': ('МОСКОВСКАЯ', 1, 1, 1),
 'ОБЛ.НИЖЕГОРОДСКАЯ': ('НИЖЕГОРОДСКАЯ', 1, 1, 1),
 'ОБЛ.РОСТОВСКАЯ': ('РОСТОВСКАЯ', 1, 1, 1),
 'ОБЛ.САРАТОВСКАЯ': ('САРАТОВСКАЯ', 1, 1, 1),
 'ОМСКАЯ': ('ОМСКАЯ', 0, 0, 0),
 'ОРЁЛ': ('ОРЁЛ', 0, 0, 0),
 'ПЕРМСКИЙ': ('ПЕРМСКИЙ', 0, 0, 0),
 'ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ': ('ПРИВОЛЖСКИЙ', 2, 2, 6),
 'РЕСП ЧУВАШСКАЯ - ЧУВАШИЯ': ('ЧУВАШСКАЯ', 1, 1, 2),
 'РЕСПУБЛИКА АЛТАЙ': ('АЛТАЙ', 1, 2, 2),
 'РЕСПУБЛИКА САХА': ('САХА', 1, 2, 2),
 'РЕСПУБЛИКАТАТАРСТАН': ('ТАТАРСТАН', 1, 2, 2),
 'РОССИЯ': ('РОССИЯ', 0, 0),
 'РОСТОВСКАЯ': ('РОСТОВСКАЯ', 0, 0, 0),
 'САХА /ЯКУТИЯ/': ('САХА', 0, 0, 0),
 'СВЕРДЛОВСКАЯ': ('СВЕРДЛОВСКАЯ', 0, 0, 0),
 'ТОМСКАЯ': ('ТОМСКАЯ', 0, 0, 0),
 'ТЮМЕНСКАЯ': ('ТЮМЕНСКАЯ', 0, 0, 0),
 'ХАКАСИЯ': ('ХАКАСИЯ', 0, 0, 0),
 'ЧУВАШСКАЯ РЕСП': ('ЧУВАШСКАЯ', 2, 1, 2),
 'ЭВЕНКИЙСКИЙ АО': ('ЭВЕНКИЙСКИЙ', 2, 1, 3),
 'АО ХАНТЫ-МАНСИЙСКИЙ-ЮГРА': ('ХАНТЫ-МАНСИЙСКИЙ', 1, 1, 3),
 'АО ЧУКОТСКИЙ': ('ЧУКОТСКИЙ', 1, 1, 3),
 'ВОЛГОГРАДСКАЯ': ('ВОЛГОГРАДСКАЯ', 0, 0, 0),
 'Г САНКТ-ПЕТЕРБУРГ': ('САНКТ-ПЕТЕРБУРГ', 1, 1, 4),
 'Г. ЧЕЛЯБИНСК': ('ЧЕЛЯБИНСК', 1, 1, 4),
 'КАМЧАТС??ИЙ КРАЙ': ('КАМЧАТСКИЙ', 2, 2, 5),
 'МОСКОВСКИЙ П': ('BAD NAME', 0, 0, 0),
 'ОБЛ. МОСКОВСКАЯ': ('МОСКОВСКАЯ', 1, 1, 1),
 'ОРЕНБУРГСКАЯ': ('ОРЕНБУРГСКАЯ', 0, 0, 0),
 'ОРЛОВСКАЯ': ('ОРЛОВСКАЯ', 0, 0, 0),
 'ПЕНЗЕНСКАЯ': ('ПЕНЗЕНСКАЯ', 0, 0, 0),
 'ПСКОВСКАЯ': ('ПСКОВСКАЯ', 0, 0, 0),
 'РЕСП.ТАТАРСТАН': ('ТАТАРСТАН', 1, 1, 2),
 'РЕСПУБЛИКА ИНГУШЕТИЯ': ('ИНГУШЕТИЯ', 1, 2, 2),
 'ЧУКОТСКИЙ АO': ('ЧУКОТСКИЙ', 2, 1, 3),
 'ЯМАЛО-НЕНЕЦКИЙ': ('ЯМАЛО-НЕНЕЦКИЙ', 0, 0, 0)}

In [7]:
class Features:
    def __init__(self, train):
        self._train = train.copy()
    
    def get_gender_feature(self, data):
        poss = ['F', 'M']
        enc = preprocessing.OneHotEncoder()
        enc.fit([[poss.index(s_gender)] for s_gender in self._train.gender])
        encode_genders = enc.transform([[poss.index(s_gender)] for s_gender in data.gender]).toarray()
        tmp = pd.DataFrame(encode_genders, 
                           columns=['gender=' + str(i) for i in range(encode_genders.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
    
    def get_age_feature(self, data):
        return data
    
    def get_marital_status_feature(self, data):
        poss = ['UNM', 'DIV', 'MAR', 'WID', 'CIV']
        enc = preprocessing.OneHotEncoder()
        enc.fit([[poss.index(s_marital_status)] for s_marital_status in self._train.marital_status])
        encode_marital_status = enc.transform([[poss.index(s_marital_status)] 
                                               for s_marital_status in data.marital_status]).toarray()
        tmp = pd.DataFrame(encode_marital_status, 
                           columns=['marital_status=' + str(i) for i in range(encode_marital_status.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
    
    def get_job_position_feature(self, data):
        poss = ['SPC', 'DIR', 'HSK', 'INV', 'WOI', 'WRK', 'ATP', 'WRP', 'UMN', 'NOR', 'PNS', 'BIS', 'INP',
                'BIU', 'ONB', 'PNA', 'PNI', 'PNV']
        enc = preprocessing.OneHotEncoder()
        enc.fit([[poss.index(s_job_position)] for s_job_position in self._train.job_position])
        encode_job_positions = enc.transform([[poss.index(s_job_position)] 
                                              for s_job_position in data.job_position]).toarray()
        tmp = pd.DataFrame(encode_job_positions, 
                           columns=['job_position=' + str(i) for i in range(encode_job_positions.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
    
    def get_credit_sum_feature(self, data):
        data.credit_sum = (data.credit_sum.apply(lambda x: x.replace(',', '.'))
                                          .astype(np.float64)
                                          .apply(lambda x: np.round(x / 1000)))
        return data
    
    def get_credit_month_feature(self, data):
        enc = preprocessing.OneHotEncoder()
        enc.fit([[credit_month] for credit_month in self._train.credit_month])
        encode_credit_months = enc.transform([[credit_month] for credit_month in data.credit_month]).toarray()
        tmp = pd.DataFrame(encode_credit_months, 
                           columns=['credit_month=' + str(i) for i in range(encode_credit_months.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
    
    def get_tariff_id_feature(self, data):
        poss = list(np.unique(self._train.tariff_id))
        enc = preprocessing.OneHotEncoder()
        enc.fit([[poss.index(tariff_id)] for tariff_id in poss])
        encode_tariff_ids = enc.transform([[poss.index(tariff_id)] for tariff_id in data.tariff_id]).toarray()
        tmp = pd.DataFrame(encode_tariff_ids, 
                           columns=['tariff_id=' + str(i) for i in range(encode_tariff_ids.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
    
    def get_score_shk_feature(self, data):
        data.score_shk = (data.score_shk.apply(lambda x: x.replace(',', '.'))
                                        .astype(np.float64))
        return data
    
    def get_education_feature(self, data):
        poss = ['SCH', 'UGR', 'GRD', 'PGR', 'ACD']
        enc = preprocessing.OneHotEncoder()
        enc.fit([[poss.index(s_education)] for s_education in self._train.education])
        encode_educations = enc.transform([[poss.index(s_education)] for s_education in data.education]).toarray()
        tmp = pd.DataFrame(encode_educations, 
                           columns=['education=' + str(i) for i in range(encode_educations.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
    
    def get_living_region_feature(self, data):
        tr_liv_reg = [cities[s_name] for s_name in self._train.living_region]
        da_liv_reg = [cities[s_name] for s_name in data.living_region]
#         tr_liv_reg = reduction_living_region(parse_living_region([s_name for s_name in self._train.living_region]))
#         da_liv_reg = reduction_living_region(parse_living_region([s_name for s_name in data.living_region]))
        poss = list(np.unique(tr_liv_reg + da_liv_reg))
        
        enc = preprocessing.OneHotEncoder()
        enc.fit([[poss.index(s_tr_liv_reg)] for s_tr_liv_reg in tr_liv_reg])
        da_liv_reg = enc.transform([[poss.index(s_living_region)]
                                     for s_living_region in da_liv_reg]).toarray()
        tr_liv_reg = enc.transform([[poss.index(s_living_region)]
                                     for s_living_region in tr_liv_reg]).toarray()
#         pca = decomposition.PCA(8)
#         pca.fit(tr_liv_reg)
#         encode_living_regions = pca.transform(da_liv_reg)
        encode_living_regions = da_liv_reg
        tmp = pd.DataFrame(encode_living_regions, 
                           columns=['living_region=' + str(i) for i in range(encode_living_regions.shape[1])],
                           index=data.index)
        data = pd.concat([data, tmp], axis=1)
        return data
        
    def get_monthly_income_feature(self, data):
        data.monthly_income = data.monthly_income.fillna(data.monthly_income.mean())
        return data
    
    def get_credit_count_feature(self, data):
        data.credit_count = data.credit_count.fillna(data.credit_count.mean())
        return data
    
    def get_overdue_credit_count_feature(self, data):
        data.overdue_credit_count = data.overdue_credit_count.fillna(data.overdue_credit_count.mean())
        return data

In [8]:
make_features = Features(train)
train = make_features.get_gender_feature(train)
train = make_features.get_age_feature(train)
train = make_features.get_marital_status_feature(train)
train = make_features.get_job_position_feature(train)
train = make_features.get_credit_sum_feature(train)
train = make_features.get_credit_month_feature(train)
train = make_features.get_tariff_id_feature(train)
train = make_features.get_score_shk_feature(train)
train = make_features.get_education_feature(train)
train = make_features.get_living_region_feature(train)
train = make_features.get_monthly_income_feature(train)
train = make_features.get_credit_count_feature(train)
train = make_features.get_overdue_credit_count_feature(train)

In [9]:
test = make_features.get_gender_feature(test)
test = make_features.get_age_feature(test)
test = make_features.get_marital_status_feature(test)
test = make_features.get_job_position_feature(test)
test = make_features.get_credit_sum_feature(test)
test = make_features.get_credit_month_feature(test)
test = make_features.get_tariff_id_feature(test)
test = make_features.get_score_shk_feature(test)
test = make_features.get_education_feature(test)
test = make_features.get_living_region_feature(test)
test = make_features.get_monthly_income_feature(test)
test = make_features.get_credit_count_feature(test)
test = make_features.get_overdue_credit_count_feature(test)

In [10]:
train.describe()

,_ID_,age,credit_sum,credit_month,tariff_id,score_shk,monthly_income,credit_count,overdue_credit_count,open_account_flg,...,living_region=258,living_region=259,living_region=260,living_region=261,living_region=262,living_region=263,living_region=264,living_region=265,living_region=266,living_region=267
count,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,...,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000,170746.000000
mean,85373.500000,36.497283,26.089209,10.980749,1.323900,0.469473,40138.288354,2.107308,0.046070,0.176028,...,0.000006,0.000726,0.004662,0.000187,0.000006,0.000826,0.010618,0.000310,0.003227,0.001886
std,49290.268869,10.546008,16.231427,3.536698,0.235813,0.124268,25044.131997,1.728427,0.205751,0.380845,...,0.002420,0.026939,0.068119,0.013689,0.002420,0.028725,0.102496,0.017616,0.056715,0.043385
min,1.000000,18.000000,3.000000,3.000000,1.000000,0.000000,5000.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42687.250000,28.000000,15.000000,10.000000,1.100000,0.379495,25000.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,85373.500000,34.000000,21.000000,10.000000,1.320000,0.461599,35000.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,128059.750000,43.000000,32.000000,12.000000,1.600000,0.552419,50000.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,170746.000000,71.000000,200.000000,36.000000,1.960000,1.128291,950000.000000,21.000000,3.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
del train['gender']
del train['marital_status']
del train['job_position']
del train['credit_month']
del train['tariff_id']
del train['education']
del train['living_region']

In [12]:
del test['gender']
del test['marital_status']
del test['job_position']
del test['credit_month']
del test['tariff_id']
del test['education']
del test['living_region']

In [13]:
train.to_csv('../data/denis_encoding_train.csv', index=False)
test.to_csv('../data/denis_encoding_test.csv', index=False)